In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [4]:
train.shape

(36992, 25)

In [5]:
train.isnull().sum().sort_values(ascending = False)

region_category                 5428
points_in_wallet                3443
preferred_offer_types            288
customer_id                        0
days_since_last_login              0
feedback                           0
complaint_status                   0
past_complaint                     0
offer_application_preference       0
used_special_discount              0
avg_frequency_login_days           0
avg_transaction_value              0
avg_time_spent                     0
internet_option                    0
last_visit_time                    0
Name                               0
medium_of_operation                0
referral_id                        0
joined_through_referral            0
joining_date                       0
membership_category                0
security_no                        0
gender                             0
age                                0
churn_risk_score                   0
dtype: int64

In [6]:
#with pd.option_context('display.max_columns', None):
    #print(train.head())

In [7]:
train[['region_category']].value_counts()

region_category
Town               14128
City               12737
Village             4699
dtype: int64

In [8]:
train.columns

Index(['customer_id', 'Name', 'age', 'gender', 'security_no',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'referral_id', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback', 'churn_risk_score'],
      dtype='object')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   36992 non-null  object 
 1   Name                          36992 non-null  object 
 2   age                           36992 non-null  int64  
 3   gender                        36992 non-null  object 
 4   security_no                   36992 non-null  object 
 5   region_category               31564 non-null  object 
 6   membership_category           36992 non-null  object 
 7   joining_date                  36992 non-null  object 
 8   joined_through_referral       36992 non-null  object 
 9   referral_id                   36992 non-null  object 
 10  preferred_offer_types         36704 non-null  object 
 11  medium_of_operation           36992 non-null  object 
 12  internet_option               36992 non-null  object 
 13  l

In [10]:
train['churn_risk_score'].value_counts()

 3    10424
 4    10185
 5     9827
 2     2741
 1     2652
-1     1163
Name: churn_risk_score, dtype: int64

In [11]:
train.dtypes

customer_id                      object
Name                             object
age                               int64
gender                           object
security_no                      object
region_category                  object
membership_category              object
joining_date                     object
joined_through_referral          object
referral_id                      object
preferred_offer_types            object
medium_of_operation              object
internet_option                  object
last_visit_time                  object
days_since_last_login             int64
avg_time_spent                  float64
avg_transaction_value           float64
avg_frequency_login_days         object
points_in_wallet                float64
used_special_discount            object
offer_application_preference     object
past_complaint                   object
complaint_status                 object
feedback                         object
churn_risk_score                  int64


In [12]:
# Find out which features is under a 'Yes' or 'No' results
train.columns[train.isin(['Yes' or 'No']).any()]

Index(['joined_through_referral', 'used_special_discount',
       'offer_application_preference', 'past_complaint'],
      dtype='object')

In [13]:
train['past_complaint'].value_counts()

No     18602
Yes    18390
Name: past_complaint, dtype: int64

In [14]:
#convert yes/no to Yes = 1, No = 0
def convert_yesno(df):
    columns = train.columns[train.isin(['Yes' or 'No']).any()]
    for i in columns:
        df[i] = df[i].replace({'Yes' : 1, 'No' :0})
    return df

In [15]:
convert_yesno(train)

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,0,xxxxxxxx,...,300.630000,53005.25,17.0,781.750000,1,1,0,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,...,306.340000,12838.38,10.0,NaN,1,0,1,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,1,CID12313,...,516.160000,21027.00,22.0,500.690000,0,1,1,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,1,CID3793,...,53.270000,25239.56,6.0,567.660000,0,1,1,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,0,xxxxxxxx,...,113.130000,24483.66,16.0,663.060000,0,1,1,Solved,Poor Website,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,fffe43004900440035003500390036003100,Cuc Tarr,46,F,6F51HFO,NaN,Basic Membership,2017-09-21,0,xxxxxxxx,...,-650.682759,27277.68,6.0,639.510000,0,1,1,No Information Available,No reason specified,4
36988,fffe43004900440033003500380036003600,Jenni Stronach,29,F,21KSM8Y,Town,Basic Membership,2016-06-27,0,xxxxxxxx,...,-638.123421,11069.71,28.0,527.990000,1,0,0,Not Applicable,Poor Customer Service,5
36989,fffe4300490044003500330034003100,Luciana Kinch,23,F,XK1IM9H,NaN,Basic Membership,2016-09-11,1,CID3838,...,154.940000,38127.56,Error,680.470000,0,1,1,Unsolved,Poor Website,4
36990,fffe43004900440031003200390039003000,Tawana Ardoin,53,M,K6VTP1Z,Village,Platinum Membership,2017-06-15,0,xxxxxxxx,...,482.610000,2378.86,20.0,197.264414,1,1,0,Not Applicable,No reason specified,3


In [16]:
train['last_visit_time'].value_counts()

07:56:41    6
02:50:54    5
21:12:56    5
17:11:53    5
22:31:06    5
           ..
10:04:00    1
09:26:11    1
06:41:43    1
10:04:38    1
17:38:16    1
Name: last_visit_time, Length: 30101, dtype: int64

In [17]:
train = train.drop(columns = ['customer_id', 'Name', 'security_no', 'referral_id', 'joining_date', 'last_visit_time'])

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           36992 non-null  int64  
 1   gender                        36992 non-null  object 
 2   region_category               31564 non-null  object 
 3   membership_category           36992 non-null  object 
 4   joined_through_referral       36992 non-null  object 
 5   preferred_offer_types         36704 non-null  object 
 6   medium_of_operation           36992 non-null  object 
 7   internet_option               36992 non-null  object 
 8   days_since_last_login         36992 non-null  int64  
 9   avg_time_spent                36992 non-null  float64
 10  avg_transaction_value         36992 non-null  float64
 11  avg_frequency_login_days      36992 non-null  object 
 12  points_in_wallet              33549 non-null  float64
 13  u

In [19]:
train.head()

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,18,F,Village,Platinum Membership,0,Gift Vouchers/Coupons,?,Wi-Fi,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,2
1,32,F,City,Premium Membership,?,Gift Vouchers/Coupons,Desktop,Mobile_Data,16,306.34,12838.38,10.0,NaN,1,0,1,Solved,Quality Customer Care,1
2,44,F,Town,No Membership,1,Gift Vouchers/Coupons,Desktop,Wi-Fi,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,5
3,37,M,City,No Membership,1,Gift Vouchers/Coupons,Desktop,Mobile_Data,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,5
4,31,F,City,No Membership,0,Credit/Debit Card Offers,Smartphone,Mobile_Data,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,5


In [20]:
list(train.select_dtypes(['object']).columns)

['gender',
 'region_category',
 'membership_category',
 'joined_through_referral',
 'preferred_offer_types',
 'medium_of_operation',
 'internet_option',
 'avg_frequency_login_days',
 'complaint_status',
 'feedback']

Do something with avg_frequency_login_days

In [34]:
train['feedback'].value_counts()

Poor Product Quality        6350
No reason specified         6290
Too many ads                6279
Poor Website                6271
Poor Customer Service       6252
Reasonable Price            1417
User Friendly Website       1391
Products always in Stock    1382
Quality Customer Care       1360
Name: feedback, dtype: int64

In [21]:
train_dummies = pd.get_dummies(train, columns = ['gender',
 'region_category',
 'membership_category',
 'joined_through_referral',
 'preferred_offer_types',
 'medium_of_operation',
 'internet_option',
 'avg_frequency_login_days',
 'complaint_status',
 'feedback'])

In [22]:
train_dummies.shape

(36992, 1702)

In [23]:
train_dummies.head()

,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,gender_F,...,complaint_status_Unsolved,feedback_No reason specified,feedback_Poor Customer Service,feedback_Poor Product Quality,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website
0,18,17,300.63,53005.25,781.75,1,1,0,2,1,...,0,0,0,0,0,1,0,0,0,0
1,32,16,306.34,12838.38,NaN,1,0,1,1,1,...,0,0,0,0,0,0,1,0,0,0
2,44,14,516.16,21027.00,500.69,0,1,1,5,1,...,0,0,0,0,1,0,0,0,0,0
3,37,11,53.27,25239.56,567.66,0,1,1,5,0,...,1,0,0,0,1,0,0,0,0,0
4,31,20,113.13,24483.66,663.06,0,1,1,5,1,...,0,0,0,0,1,0,0,0,0,0


In [24]:
list(train_dummies.select_dtypes(['object']).columns)

[]

feature engineering
1. number of years as members (from join date)
2. Pos/Neg feedback (feedback)
3. Day, afternoon or night of last login (last_visit_time)
4. teens, adults, or seniors based on age (age)
5. Referral (Yes/No)